In [2]:
#### Download Training text data for topic modelling

!wget https://raw.githubusercontent.com/vinid/data/master/italian_documents.txt
!wget https://raw.githubusercontent.com/vinid/data/master/dbpedia_sample_abstract_20k_unprep.txt


--2021-06-14 11:53:55--  https://raw.githubusercontent.com/vinid/data/master/italian_documents.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7122 (7.0K) [text/plain]
Saving to: ‘italian_documents.txt.1’

italian_documents.t 100%[===================>]   6.96K  --.-KB/s    in 0.001s  

2021-06-14 11:53:56 (12.5 MB/s) - ‘italian_documents.txt.1’ saved [7122/7122]

--2021-06-14 11:53:56--  https://raw.githubusercontent.com/vinid/data/master/dbpedia_sample_abstract_20k_unprep.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62

In [5]:
####we will still use the non pre-processed dataset to generate the representations from distiluse-base-multilingual-cased model.

In [3]:
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
import nltk 

nltk.download('stopwords') # stopwrods to be removed

text_file = "dbpedia_sample_abstract_20k_unprep.txt" # the file that contains our docuemtns

documents = [line.strip() for line in open(text_file, encoding="utf-8").readlines()] # we load the documents

# simple preprocessing that removes stopwords and punctuation
sp = WhiteSpacePreprocessing(documents, stopwords_language='english') 

# this function returns the pre and the unpre processed documents and a vocab with the most frequent 2K tokens
# these tokens are going to be used to represent the topics
preprocessed_documents, unpreprocessed_documents, vocab = sp.preprocess() 

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/abhishek/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
from contextualized_topic_models.models.ctm import ZeroShotTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation

tp = TopicModelDataPreparation("distiluse-base-multilingual-cased") # instantiate a contextualized model

# # create the training set
training_dataset = tp.fit(text_for_contextual=unpreprocessed_documents, text_for_bow=preprocessed_documents)



Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


In [9]:
# train the model
# input_size is the size of our vocabulary
# bert_input_size is the size of the embedding that comes from the contextualized model
# n_components is the number of topic we are going to get from the model
# n_epochs is the number of epochs we are going to train our model for
ctm = ZeroShotTM(bow_size=len(tp.vocab), contextual_size=512, n_components=50, num_epochs=100)
ctm.fit(training_dataset) # run the model

Epoch: [100/100]	 Seen Samples: [2000000/2000000]	Train Loss: 133.81680244140625	Time: 0:00:05.208293: : 100it [08:32,  5.12s/it]


In [10]:
ctm.get_topic_lists(5)

[['journal', 'professor', 'university', 'history', 'studies'],
 ['war', 'class', 'ship', 'navy', 'royal'],
 ['iran', 'persian', 'province', 'families', 'rural'],
 ['directed', 'film', 'silent', 'stars', 'produced'],
 ['usually', 'different', 'often', 'form', 'planet'],
 ['cup', 'final', 'championship', 'tournament', 'teams'],
 ['club', 'stadium', 'association', 'based', 'home'],
 ['king', 'son', 'iii', 'bishop', 'duke'],
 ['point', 'island', 'land', 'coast', 'islands'],
 ['award', 'film', 'awards', 'indian', 'best'],
 ['greek', 'bc', 'egypt', 'al', 'modern'],
 ['album', 'released', 'studio', 'second', 'band'],
 ['race', 'grand', 'held', 'prix', 'open'],
 ['school', 'high', 'secondary', 'girls', 'students'],
 ['air', 'services', 'force', 'airport', 'company'],
 ['canadian', 'former', 'professional', 'hockey', 'player'],
 ['civil', 'parish', 'england', 'community', 'small'],
 ['book', 'published', 'comic', 'comics', 'manga'],
 ['complete', 'county', 'list', 'places', 'properties'],
 ['su

In [11]:
 lda_vis_data = ctm.get_ldavis_data_format(tp.vocab, training_dataset,n_samples=10)

Sampling: [10/10]: : 10it [00:17,  1.73s/it]


In [14]:
# !pip install pyLDAvis

In [15]:
import pyLDAvis as vis
movies_pd = vis.prepare(**lda_vis_data)
vis.display(movies_pd)

In [16]:
#preditctions
topics_predictions = ctm.get_thetas(training_dataset, n_samples=5)

Sampling: [5/5]: : 5it [00:08,  1.76s/it]


In [17]:
preprocessed_documents[0] # see the text of our preprocessed document

'mid peninsula highway proposed across peninsula canadian province ontario although highway connecting hamilton fort south international study published ministry'

In [19]:
import numpy as np

In [20]:
topic_number = np.argmax(topics_predictions[0]) # get the topic id of the first document

In [21]:
ctm.get_topic_lists(5)[topic_number] #and the topic should be about natural location related things

['line', 'railway', 'road', 'rail', 'station']